In [27]:
import pyodbc
import pandas as pd 
import requests
import geocoder
from geopy.geocoders import Nominatim
from IPython.display import clear_output
from pprint import pprint
import time
import re
import numpy as np

In [11]:
cnxn = pyodbc.connect('DSN=SQLAmlin;UID=MSAmlin;PWD=password')

In [3]:
cur = cnxn.cursor()

In [4]:
def nlp_col(df):
    names = []
    lens = []
    for d in df.columns:
        if df[d].dtype == 'O':
            names.append(d)
            lens.append(df[d].str.len().mean())

    return names[lens.index(max(lens))]

In [91]:
def addr_from_text(test_text):
    road_words = ['yard', 'walk', 'hill', 'triangle', 'dale', 'passage', \
                 'lane', 'rise', 'grove', 'avenue', 'retreat', 'gardens', \
                 'terrace', 'arcade', 'court', 'villas', 'square', 'way', \
                 'cottages', 'close', 'boulevard', 'end', 'avenue', 'drive', \
                 'place', 'row', 'mews', 'street', 'road']
    try:
        test_text = [word for word in test_text.split()]
        for i in range(len(test_text)):
            #print(i)
            if test_text[i].replace(',', '').lower() in road_words:
                surr_words = [test_text[i-2], test_text[i-1], test_text[i], test_text[i+1], test_text[i+2], test_text[i+3]]
                
                surr_words = [w for w in surr_words if w.istitle()]
                
                # takes out words after a full stop - currently would go from full stops before the 
                for i in range(len(surr_words)):
                    if surr_words[i].find('.') != -1:
                        surr_words = surr_words[:i+1]
                        break
                
                surr_words = [re.search("[a-zA-Z]+", w).group() for w in surr_words]
                #print(surr_words)
            
                # takes the first address that fits the format to avoid returning empty strings
                if len(surr_words) > 0:
                    break
        if ' '.join(surr_words) == '':
            return 'None'
        else:
            return ' '.join(surr_words)#poss_addrs
    
    except IndexError:
        return 'None'
    except:
        return 'None'

In [92]:
def geotext_address(text):
    from geotext import GeoText #pip install geotext

    roadarray = ['yard', 'walk', 'hill', 'triangle', 'dale', 'passage', \
                 'lane', 'rise', 'grove', 'avenue', 'retreat', 'gardens', \
                 'terrace', 'arcade', 'court', 'villas', 'square', 'way', \
                 'cottages', 'close', 'boulevard', 'end', 'avenue', 'drive', \
                 'place', 'row', 'mews', 'street', 'road']
    # array of locational keywords

    #interates through the numbered keys of text dictionary
    
    address = {} # initialises address dictionary
    for word in roadarray: # iterates through each word in keywords
        try:
            search  = r'((?:\w+(?:\s+|$)){1})' + word # search for the word before the key word iterated
            road = re.search(search, text, re.IGNORECASE) #  assigns var raod to the search query ignoring case
            address['ROAD'] = road.group() # address['ROAD'] is the word before keyword+keyword
        except:
            pass

    # If address['ROAD'] still doesn't exist, assign it as UNKNOWN   
    try:
        address['ROAD']
    except:
        address['ROAD'] = 'UNKNOWN'

    # Assigns TOWN as the geotext identified city
    try:
        address['TOWN'] = GeoText(text).cities[-1] # The last entry of cities is the most granular
    except:   
        address['TOWN'] = 'UNKNOWN' # Sets it to null value if town didnt match anything
    
    address # appends the address dictionary into the final locdic
    #return address['ROAD'] + ' ' + address['TOWN']
    #locdf = pd.DataFrame(address)#.from_dict(address, orient='index') # transforms locdic into a df
    address_string = address['ROAD'] + ' '+ address['TOWN']
    return address_string

In [57]:
nw_query = 'select * from [initial].[NorthWales]'

In [141]:
nw_df = pd.read_sql(nw_query, cnxn)

In [142]:
#matt first
text_col = nlp_col(nw_df)
nw_df['LookupAddress'] = nw_df[text_col].apply(addr_from_text)
nw_df['GeotextLookupAddress'] = nw_df[text_col].apply(geotext_address)
nw_df['LookupAddress'] = nw_df['LookupAddress'].replace('None', nw_df['GeotextLookupAddress'])
nw_df = nw_df.drop('GeotextLookupAddress', axis=1)

In [128]:
text_col

'3text'

In [143]:
nw_df

,IncidentID,1dateposted,2title,3text,LookupAddress
0,1,Posted 01/01/2012 14:10:36,Fatal Fire Llandudno,"North Wales Police have confirmed that a man, ...",Mostyn Street Llandudno
1,2,Posted 01/02/2018 15:13:45,All Wales Fire Investigation Training Forum,"On the 24th and 25th of January 2018, the thre...",Detective Sergeant Dale Scriven Dyfed Powys
2,3,Posted 01/03/2014 14:10:36,Fire at Bangor school,Firefighters were called to Ysgol Ein Harglwyd...,UNKNOWN Bangor
3,4,Posted 01/05/2018 15:43:40,Fire in Sandycroft,Crews are in attendance at a fire in Factory R...,Factory Road Sandycroft
4,5,Posted 01/05/2018 17:38:38,Fire in Sandycroft - update,Crews are in attendance at a fire in Factory R...,Factory Road Sandycroft
5,6,Posted 01/05/2018 18:03:22,Fire in Sandycroft - update 1,Crews are in attendance at a fire in Factory R...,Factory Road Sandycroft
6,7,Posted 01/08/2014 14:10:36,Man dies after garden outbuilding fire in Llan...,A man has died following a fire involving a ga...,UNKNOWN Police
7,8,Posted 01/08/2018 08:31:09,"Fire at Kronospan, near Wrexham",Firefighters from North Wales Fire and Rescue ...,UNKNOWN Oswestry
8,9,Posted 02/01/2014 14:10:36,Importance of smoke alarms highlighted after R...,A senior fire officer is highlighting the impo...,UNKNOWN UNKNOWN
9,10,Posted 02/02/2012 14:10:36,Suspected Bottled Gas Explosion in Morfa Bychan,A man has been taken to hospital with superfic...,under way Bangor


In [47]:
data = nw_df
row_count = data.shape[0]
none_count = data[data['Address'] == 'None'].shape[0]
success = str(round(100*((row_count-(none_count))/row_count), 2)) + '%'

print('Success Report \nRows:{}\nNones:{}\nSuccess rate: {}'.format(row_count,none_count, success))

Success Report 
Rows:143
Nones:86
Success rate: 39.86%


In [37]:
nw_df.iloc[141]['Address']

nan

In [140]:
data = nw_df
row_count = data.shape[0]
none_count = data[data['Address'] == 'None'].shape[0]
unkown_count = data[data['Address'] == 'UNKNOWN UNKNOWN'].shape[0]
success = str(round(100*((row_count-(none_count+unkown_count))/row_count), 2)) + '%'

print('Success Report \nRows:{}\nNones:{}\nUnknowns:{}\nSuccess rate: {}'.format(row_count,none_count,unkown_count, success))

Success Report 
Rows:143
Nones:0
Unknowns:24
Success rate: 83.22%
